In [1]:
# Dependencies
from bs4 import BeautifulSoup
import requests
import pymongo
from splinter import Browser


In [2]:
#for splinter
#executable_path = {'executable_path': '/usr/local/bin/chromedriver'}
#browser = Browser('chrome', **executable_path, headless=False)

In [3]:
# Initialize PyMongo to work with MongoDBs
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)

In [4]:
# Define database and collection
db = client.restaurants_db
collection = db.find_me_gluten_free

In [5]:
# URL of page to be scraped
url = 'https://www.findmeglutenfree.com/search?lat=35.1982836&lng=-111.65130199999999&q=&a=flagstaff%2C+az&local=t'

# Retrieve page with the requests module
response = requests.get(url)

# Create BeautifulSoup object; parse with 'lxml'
soup = BeautifulSoup(response.text, 'lxml')

In [6]:
#or browser
#browser.visit(url)
#for x in range(1, 100):

 #   html = browser.html
  #  soup = BeautifulSoup(html, 'html.parser')

In [7]:
# Examine the results, then determine element that contains sought info
# results are returned as an iterable list
results = soup.find_all('li', class_='mt-4')

In [8]:
# Loop through returned results
links=[]
for result in results:
    try: 
        Name = result.find('a', class_='align-middle').text
        Address = result.find('span' , class_='sl-addr mt-2').text
        link = "https://www.findmeglutenfree.com" + result.a['href']
        links.append(link)
        
        #for link in links:
            
        response_ = requests.get(link)
        soup_ = BeautifulSoup(response_.text, 'lxml')
        phone = soup_.find('div', class_='mt-3 font-weight-bold').text
        web= soup_.find('div', class_='mt-3 mr-3').text
        print('-------------')
        print(Name)
        print(Address)
        print(link)
        print(phone)
        print(web)
                        
        

        # Dictionary to be inserted as a MongoDB document
        post = {
                            'Name': Name,
                            'Address': Address, 
                            'url': link,
                            'phone number': phone,
                            'website': web
                
                            }

        collection.insert_one(post)
    except Exception as e:
        print(e)      
        
                   

-------------
Picazzo's Healthy Italian Kitchen
1855 Arizona 89A, Sedona, AZ 86336
https://www.findmeglutenfree.com/biz/picazzos-healthy-italian-kitchen/2203008?sp=t
(928) 282-4140

picazzos.com

-------------
diablo burger
120 N Leroux St, Flagstaff, AZ 86001
https://www.findmeglutenfree.com/biz/diablo-burger/1785014
(928) 774-3274

diabloburger.com

-------------
The Toasted Owl Cafe
12 S Mikes Pike St, Flagstaff, AZ 86001
https://www.findmeglutenfree.com/biz/the-toasted-owl-cafe/6461896180629504
(928) 774-5326

thetoastedowl.com

-------------
Mix Flagstaff
120 N Leroux St, Flagstaff, AZ 86001
https://www.findmeglutenfree.com/biz/mix-flagstaff/7011596
(928) 774-8200

mixflagstaff.com

-------------
Karma Sushi Bar Grill
6 E Route 66, Flagstaff, AZ 86001
https://www.findmeglutenfree.com/biz/karma-sushi-bar-grill/4513297772576768
(928) 774-6100

karmaflagstaff.com

-------------
Beaver Street Brewery
11 Beaver St #1, Flagstaff, AZ 86001
https://www.findmeglutenfree.com/biz/beaver-stre

-------------
Oregano's Pizza Bistro
980 N Country Club Dr, Flagstaff, AZ 86004
https://www.findmeglutenfree.com/biz/oreganos-pizza-bistro/4756622853406720
(928) 233-3000

oreganos.com

-------------
The Toasted Owl - East Side
5200 E Cortland Blvd, Flagstaff, AZ 86004
https://www.findmeglutenfree.com/biz/the-toasted-owl-east-side/5102945905541120
(928) 526-0610

thetoastedowl.com

-------------
Indian Gardens Cafe & Market
3951 N State Rte 89A, Sedona, AZ 86336
https://www.findmeglutenfree.com/biz/indian-gardens-cafe-and-market/4723397963677696
(928) 282-7702

indiangardens.com

-------------
Tii Gavo at Enchantment Resort
525 Boynton Canyon Road, Sedona, AZ 86336
https://www.findmeglutenfree.com/biz/tii-gavo-at-enchantment-resort/5459501942571008
(844) 244-9489

enchantmentresort.com

-------------
Che Ah Chi
525 Boynton Canyon Rd, Sedona, AZ 86336
https://www.findmeglutenfree.com/biz/che-ah-chi/6522957121454080
(928) 204-6000

enchantmentresort.com



In [9]:
# Display items in MongoDB collection
listings = collection.find()

for listing in listings:
    print(listing)

{'_id': ObjectId('5d626ac4ea80e37e77692f83'), 'Name': "Picazzo's Healthy Italian Kitchen", 'Address': '7325 E Frank Lloyd Wright Blvd, Scottsdale, AZ 85260', 'url': 'https://www.findmeglutenfree.com/biz/picazzos-healthy-italian-kitchen/2291004?sp=t', 'phone number': '(480) 990-2212', 'website': '\npicazzos.com\n'}
{'_id': ObjectId('5d626ac7ea80e37e77692f84'), 'Name': 'The Counter', 'Address': '50 N Central Ave, Phoenix, AZ 85004', 'url': 'https://www.findmeglutenfree.com/biz/the-counter/6679924027162624', 'phone number': '(602) 466-3411', 'website': '\nthecounter.com\n'}
{'_id': ObjectId('5d626ac7ea80e37e77692f85'), 'Name': 'Cibo', 'Address': '603 N 5th Ave, Phoenix, AZ 85003', 'url': 'https://www.findmeglutenfree.com/biz/cibo/13365163', 'phone number': '(602) 441-2697', 'website': '\ncibophoenix.com\n'}
{'_id': ObjectId('5d626ac8ea80e37e77692f86'), 'Name': 'The Strand', 'Address': '2 E Jefferson St #113, Phoenix, AZ 85005', 'url': 'https://www.findmeglutenfree.com/biz/the-strand/60139

In [11]:
#write to csv
import csv
with open('flagstaff.csv', mode='w') as f:  # Just use 'w' mode in 3.x\
    
    w = csv.writer(f,listings)
   
    